In [46]:
# Se importan las librerías

# Librerías estándar
import os
import subprocess
import pprint
import time
import json
from datetime import datetime
import base64
import locale
import os.path
import email
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import re

# Librerías de terceros
import pandas as pd
import numpy as np
from docxtpl import DocxTemplate
import docx2pdf as pdf
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2 import service_account
from google.oauth2.credentials import Credentials
from google.oauth2.service_account import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload


In [47]:
# Se genera la autentificación de google

key = "/path/to/your/keyfile.json"
SCOPES = ["https://www.googleapis.com/auth/drive"]
creds = service_account.Credentials.from_service_account_file(key, scopes=SCOPES)
service = build("drive", "v3", credentials=creds)

# Acuerdos de Registro

In [48]:
# Se crea un patrón para que se eliminen siempre los archivos word y PDF en las carpetas

carpeta = "/path/to/your/Acuerdos_Victimas"

if os.path.exists(carpeta):
    for elemento in os.listdir(carpeta):
        elemento_path = os.path.join(carpeta, elemento)
        if os.path.isfile(elemento_path) or os.path.islink(elemento_path):
            os.remove(elemento_path)
        elif os.path.isdir(elemento_path):
            shutil.rmtree(elemento_path)

print(f"Todos los elementos de la carpeta '{carpeta}' han sido eliminados.")

carpeta1 = "/path/to/your/PDF_Acuerdos_Victimas"

if os.path.exists(carpeta1):
    for elemento1 in os.listdir(carpeta1):
        elemento_path1 = os.path.join(carpeta1, elemento1)
        if os.path.isfile(elemento_path1) or os.path.islink(elemento_path1):
            os.remove(elemento_path1)
        elif os.path.isdir(elemento_path1):
            shutil.rmtree(elemento_path1)

print(f"Todos los elementos de la carpeta '{carpeta1}' han sido eliminados.")

Todos los elementos de la carpeta '/path/to/your/Acuerdos_Victimas' han sido eliminados.
Todos los elementos de la carpeta '/path/to/your/PDF_Acuerdos_Victimas' han sido eliminados.


In [49]:
# Se carga la base de datos

data = pd.read_csv("data_registro.csv", encoding="UTF-8", index_col="index_número_registro")

In [50]:
# Se modifican columnas

data["número_registro"] = data.index
columnas = data.columns.tolist()
columnas.insert(3, columnas.pop(columnas.index("número_registro")))
data = data[columnas]


In [51]:
# Se establece la fecha de hoy

locale.setlocale(locale.LC_TIME, "es_ES")

fecha_hoy = datetime.now()
fecha_hoy = fecha_hoy.strftime("%d de %B de %Y")

locale.setlocale(locale.LC_ALL, "")

'C/UTF-8/C/C/C/C'

In [52]:
# Se filtran los datos para cada acuerdo particular

delitos = data[(data["tipo_registro"] == "Delito – MP") & (data["fecha_registro"] == fecha_hoy)]
delitos = delitos.iloc[:, [0, 1, 2, 3, 4, 5, 6, 7, 8]]

dh = data[(data["tipo_registro"] == "Violación DH") & (data["fecha_registro"] == fecha_hoy)]
dh = dh.iloc[:, [0, 2, 3, 5, 6, 9, 10, 11, 12, 13, 14]]

judiciales = data[(data["tipo_registro"] == "Delito – Judicial") & (data["fecha_registro"] == fecha_hoy)]
judiciales = judiciales.iloc[:, [0, 1, 2, 3, 4, 5, 6, 15, 16]]

In [53]:
# Se crea un patrón para generar la columna Alfanúmericaregistro

def generar_alfanumerica_registro(row):
    if pd.notna(row["número_registro"]):
        return f"CEAVI/RELOVI/{row["número_registro"]}/2024"
    else:
        return None

data["AlfanúmericaRegistro"] = data.apply(generar_alfanumerica_registro, axis=1)


In [54]:
# Se introduce el código para delitos

for index, row in delitos.iterrows():
    doc = DocxTemplate("FormatoRegistro_Delitos_MP.docx")
    registros = {
        "núcleo_víctima": row["núcleo_víctima"],
        "nombre_víctima": row["nombre_víctima"],
        "número_registro": row["número_registro"],
        "tipo_delito": row["tipo_delito"],
        "fecha_registro": row["fecha_registro"],
        "fecha_acuerdo_ministerial": row["fecha_acuerdo_ministerial"],
        "carpeta_investigacion": row["carpeta_investigacion"],
        "directa_indirecta": row["directa_indirecta"]
                }
    doc.render(registros)
    doc.save(f"/path/to/your/Acuerdos_Victimas/AcuerdoRegistro_{index}.docx")

In [55]:
# Se introduce el código para violaciones a derechos humanos

for index, row in dh.iterrows():
    doc = DocxTemplate("Formato_Registro_Violaciones_Recos.docx")
    registros = {
        "nombre_víctima": row["nombre_víctima"],
        "clave_víctima": row["clave_víctima"],
        "número_registro": row["número_registro"],
        "fecha_registro": row["fecha_registro"],
        "numero_anexo": row["numero_anexo"],
        "numero_recomendacion": row["numero_recomendacion"],
        "nombre_recomendacion": row["nombre_recomendacion"],
        "fecha_emision_recomendacion": row["fecha_emision_recomendacion"],
        "directa_indirecta": row["directa_indirecta"],
        "derechos_violados": row["derechos_violados"]
                }
    doc.render(registros)
    doc.save(f"/path/to/your/Acuerdos_Victimas/AcuerdoRegistro_{index}.docx")

In [56]:
# Se introduce el código para judiciales


for index, row in judiciales.iterrows():
    doc = DocxTemplate("FormatoRegistro_Delitos_Jueces.docx")
    registros = {
        "núcleo_víctima": row["núcleo_víctima"],
        "nombre_víctima": row["nombre_víctima"],
        "número_registro": row["número_registro"],
        "tipo_delito": row["tipo_delito"],
        "fecha_registro": row["fecha_registro"],
        "fecha_sentencia": row["fecha_sentencia"],
        "expediente_judicial": row["expediente_judicial"],
        "directa_indirecta": row["directa_indirecta"]
                }
    doc.render(registros)
    doc.save(f"/path/to/your/acuerdos_registro/Acuerdos_Víctimas_Python/AcuerdoRegistro_{index}.docx")

In [57]:
# Se convierten los archivos word en PDF

pdf.convert("/path/to/your/acuerdos_registro/Acuerdos_Víctimas_Python", "/path/to/your/acuerdos_registro/PDF_Acuerdos_Víctimas_Python")

100%|███████████████████████████████████████████| 28/28 [00:28<00:00,  1.03s/it]


# Oficios de informe a autoridades

In [58]:
# Se crea un patrón para que se eliminen siempre los archivos word en la carpeta

carpeta2 = "/path/to/your/acuerdos_registro/Word_Oficios"

if os.path.exists(carpeta2):
    for elemento2 in os.listdir(carpeta2):
        elemento_path2 = os.path.join(carpeta2, elemento2)
        if os.path.isfile(elemento_path2) or os.path.islink(elemento_path2):
            os.remove(elemento_path2)
        elif os.path.isdir(elemento_path2):
            shutil.rmtree(elemento_path2)

print(f"Todos los elementos de la carpeta '{carpeta2}' han sido eliminados.")

carpeta3 = "/path/to/your/acuerdos_registro/PDF_Oficios"

if os.path.exists(carpeta3):
    for elemento3 in os.listdir(carpeta3):
        elemento_path3 = os.path.join(carpeta3, elemento3)
        if os.path.isfile(elemento_path3) or os.path.islink(elemento_path3):
            os.remove(elemento_path3)
        elif os.path.isdir(elemento_path3):
            shutil.rmtree(elemento_path3)

print(f"Todos los elementos de la carpeta '{carpeta3}' han sido eliminados.")


Todos los elementos de la carpeta '/path/to/your/acuerdos_registro/Word_Oficios' han sido eliminados.
Todos los elementos de la carpeta '/path/to/your/acuerdos_registro/PDF_Oficios' han sido eliminados.


In [59]:
# Se filtran los datos

data = data[data["fecha_registro"] == fecha_hoy]

In [60]:
# Se modifica la estructura de datos para los oficios dirigidos a MPs, todos los núcleos familiares pasan a una sola columna separados por una coma

of_delitos = data[data["tipo_registro"] == "Delito – MP"]
of_delitos = data.groupby(["tipo_registro", "núcleo_víctima", "carpeta_investigacion", "fecha_registro"]).agg({
    "nombre_víctima": ", ".join,
    "AlfanúmericaRegistro": ", ".join
}).reset_index()


In [61]:
# Se modifica la estructura de datos para los oficios dirigidos a jueces

of_judiciales = data[data["tipo_registro"] == "Delito – Judicial"]
of_judiciales = data.groupby(["tipo_registro", "núcleo_víctima", "expediente_judicial", "fecha_registro"]).agg({
    "nombre_víctima": ", ".join,
    "AlfanúmericaRegistro": ", ".join
}).reset_index()

In [62]:
# Se modifica la estructura de datos para los oficios dirigidos a la CDHCM

of_dh = data[data["tipo_registro"] == "Violación DH"]
of_dh = data.groupby(["tipo_registro", "numero_anexo", "numero_recomendacion", "fecha_registro"]).agg({
    "nombre_víctima": ", ".join,
    "AlfanúmericaRegistro": ", ".join
}).reset_index()

In [63]:
# Se concatenan los tres últimos dataframes, renombran las columnas y establece un nuevo orden de estas

data1 = [of_delitos, of_dh, of_judiciales]
data1 = pd.concat(data1, ignore_index=True, sort=False)

data1.rename(columns={
    "fecha_registro": "fecha_oficio",
    "nombre_víctima": "nombres_víctimas",
    "AlfanúmericaRegistro": "número_registro",
    "carpeta_investigacion": "número_carpeta",
    "numero_recomendacion": "número_recomendación"
}, inplace=True)

new_columns = [
    "index_núm_oficio", 
    "nombre_agente", 
    "cargo_agente", 
    "fiscalía_especializada", 
    "nombre_juez", 
    "cargo_juez",
    "correo_electrónico"
]

for col in new_columns:
    data1[col] = np.nan

nuevo_orden = [
    "index_núm_oficio",
    "tipo_registro",
    "fecha_oficio",
    "núcleo_víctima",
    "nombres_víctimas",
    "número_registro",
    "número_carpeta",
    "nombre_agente",
    "cargo_agente",
    "fiscalía_especializada",
    "número_recomendación",
    "numero_anexo",
    "expediente_judicial",
    "nombre_juez",
    "cargo_juez",
    "correo_electrónico"
]

data1 = data1[nuevo_orden]
data1 = data1.fillna("NA")

filtro_feidf = data1["número_carpeta"].str.contains("FEIDF", na=False)

data1.loc[filtro_feidf, "nombre_agente"] = "LIC. ..."
data1.loc[filtro_feidf, "cargo_agente"] = "FISCAL"
data1.loc[filtro_feidf, "fiscalía_especializada"] = "FISCALÍA DE INVESTIGACIÓN DEL DELITO DE ..."
data1.loc[filtro_feidf, "correo_electrónico"] = "...@....com"

data1.loc[data1["tipo_registro"] == "Violación DH", "correo_electrónico"] = "...@...org.mx"


data1["Asunto"] = data1["nombres_víctimas"].apply(lambda x: f"Informe Registro {x}")


In [64]:
# Se definen las liberías y servicios para solicitar la autentificación en Google, tomar los números de oficios y llenar la base del control de gestión

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('/path/to/your/keyfile.json', scope)
client = gspread.authorize(creds)

spreadsheet = client.open_by_url("https://docs.google.com/spreadsheets/d/1Rb2rhVQ5n9SGiRnwDFJdi-O0efDuR8ifu6hvT9tw3kE/edit?gid=0#gid=0")
worksheet = spreadsheet.worksheet("Oficios_Salida")

In [65]:
# Se copian los números de oficio

data2 = worksheet.get_all_records()
df = pd.DataFrame(data2)

columna_asunto = worksheet.col_values(5)
fila_inicio = len(columna_asunto) + 1

max_rows = 1000
if fila_inicio > max_rows:
    fila_inicio = max_rows

numero_oficio_values = worksheet.col_values(3)[fila_inicio-1: fila_inicio-1+len(df)]  # Extraer los valores desde la fila de inicio

if len(numero_oficio_values) < len(df):
    print(f"Advertencia: La longitud de 'Número_Oficio' ({len(numero_oficio_values)}) es menor que la longitud de 'df' ({len(df)}). Ajustando longitud.")
    df = df.head(len(numero_oficio_values))

data1_length = len(data1)
numero_oficio_values_length = len(numero_oficio_values)
if numero_oficio_values_length > data1_length:
    print(f"Advertencia: La longitud de 'Número_Oficio' ({numero_oficio_values_length}) es mayor que la longitud de 'data1' ({data1_length}). Ajustando longitud.")
    numero_oficio_values = numero_oficio_values[:data1_length]

data1.iloc[:len(numero_oficio_values), data1.columns.get_loc("index_núm_oficio")] = numero_oficio_values

Advertencia: La longitud de 'Número_Oficio' (171) es menor que la longitud de 'df' (999). Ajustando longitud.
Advertencia: La longitud de 'Número_Oficio' (171) es mayor que la longitud de 'data1' (8). Ajustando longitud.


In [66]:
# Se llena la base de drive de control de gestión en la columna de asunto

data2 = worksheet.get_all_records()
df = pd.DataFrame(data2)

asunto_col = worksheet.col_values(5)
first_empty_row = len(asunto_col) + 1

updates = []
for idx, row in data1.iterrows():
    row_index = first_empty_row + idx
    updates.append({
        "range": f"E{row_index}",
        "values": [[row["Asunto"]]]
    })

try:
    worksheet.batch_update(updates)
    print("Datos actualizados correctamente.")
except Exception as e:
    print(f"Error al actualizar los datos: {e}")

Datos actualizados correctamente.


In [67]:
# Se crea un CSV, esto para que los algunos datos son necesarios que se introduzcan manualmente

data1.to_csv("Oficios_Informe.csv", index=False)

In [68]:
# Se abre el CSV creado por el programa predeterminado, se ingresan nuevos datos, se cierra y sigue corriendo el código

try:
    process = subprocess.Popen(["open", "Oficios_Informe.csv"])
    process.wait()
    print("El archivo ha sido cerrado. Continuando con la ejecución del código.")
except FileNotFoundError:
    print(f"Error: No se pudo encontrar el archivo 'Oficios_Informe.csv'")

El archivo ha sido cerrado. Continuando con la ejecución del código.


In [69]:
# Se cargan de nueva cuenta los dstos del CSV

data1 = pd.read_csv("Oficios_Informe.csv", encoding="UTF-8", index_col="index_núm_oficio")

In [70]:
data1["número_oficio"] = data1.index
columnas = data1.columns.tolist()
columnas.insert(3, columnas.pop(columnas.index("número_oficio")))
data1 = data1[columnas]

In [71]:
# Se filtran los datos para cada oficio particular

of_delitos = data1[(data1["tipo_registro"] == "Delito – MP") & (data1["fecha_oficio"] == fecha_hoy)]

of_dh = data1[(data1["tipo_registro"] == "Violación DH") & (data1["fecha_oficio"] == fecha_hoy)]

of_judiciales = data1[(data1["tipo_registro"] == "Delito – Judicial") & (data1["fecha_oficio"] == fecha_hoy)]


In [72]:
# Se introduce el código para oficios de delitos

for index, row in of_delitos.iterrows():
    doc1 = DocxTemplate("FormatoOficio_Notificación_MP.docx")
    registros1 = {
        "fecha_oficio": row["fecha_oficio"],
        "número_oficio": row["número_oficio"],
        "nombres_víctimas": row["nombres_víctimas"],
        "número_registro": row["número_registro"],
        "número_carpeta": row["número_carpeta"],
        "nombre_agente": row["nombre_agente"],
        "cargo_agente": row["cargo_agente"],
        "fiscalía_especializada": row["fiscalía_especializada"]
                }
    doc1.render(registros1)
    doc1.save(f"/path/to/your/acuerdos_registro/Word_Oficios/OficioRELOVI_{index}.docx")

In [73]:
# Se introduce el código para oficios de violaciones a los derechos humanos

for index, row in of_dh.iterrows():
    doc1 = DocxTemplate("FormatoOficio_Notificación_CDHCM.docx")
    registros1 = {
        "fecha_oficio": row["fecha_oficio"],
        "número_oficio": row["número_oficio"],
        "nombres_víctimas": row["nombres_víctimas"],
        "número_registro": row["número_registro"],
        "número_recomendación": row["número_recomendación"]
                }
    doc1.render(registros1)
    doc1.save(f"/path/to/your/acuerdos_registro/Word_Oficios/OficioRELOVI_{index}.docx")

In [74]:
# Se introduce el código para oficios de delitos judiciales

for index, row in of_judiciales.iterrows():
    doc1 = DocxTemplate("FormatoOficio_Notificación_Jueces.docx")
    registros1 = {
        "fecha_oficio": row["fecha_oficio"],
        "número_oficio": row["número_oficio"],
        "nombres_víctimas": row["nombres_víctimas"],
        "número_registro": row["número_registro"],
        "expediente_judicial": row["expediente_judicial"],
        "nombre_juez": row["nombre_juez"],
        "cargo_juez": row["cargo_juez"],
                }
    doc1.render(registros1)
    doc1.save(f"/path/to/your/acuerdos_registro/Word_Oficios/OficioRELOVI_{index}.docx")

In [75]:
# Se convierten los oficios en PDF

pdf.convert("/path/to/your/acuerdos_registro/Word_Oficios", "/path/to/your/acuerdos_registro/PDF_Oficios")

100%|█████████████████████████████████████████████| 8/8 [00:12<00:00,  1.56s/it]


# Subir archivos a Google Drive

In [76]:
# Se definen las rutas locales y de Google Drive

ruta_origen = "PDF_Acuerdos_Víctimas_Python"
ruta_destino = "1y..._X"


In [77]:
# Se crea la función para copiar los archivos de la carpeta local a Google Drive

def mac_to_drive(ruta_origen, ruta_destino):
    for nombre_archivo in os.listdir(ruta_origen):
        ruta_archivo = os.path.join(ruta_origen, nombre_archivo)
        if os.path.isfile(ruta_archivo):
            media = MediaFileUpload(ruta_archivo, resumable=True)
            archivo_metadata = {
                "name": nombre_archivo,
                "parents": [ruta_destino]
            }
            file = service.files().create(
                body=archivo_metadata,
                media_body=media,
                fields="id"
            ).execute()
            print(f"Archivo {nombre_archivo} subido con ID: {file.get("id")}")

mac_to_drive(ruta_origen, ruta_destino)


Archivo AcuerdoRegistro_788.pdf subido con ID: 1-m-QBScWSXxtyyrgWhLoGjEtwV9Fcziu
Archivo AcuerdoRegistro_789.pdf subido con ID: 1igzJ_9-NCpXpaeSh5D9t91ZUZn9-nL97
Archivo AcuerdoRegistro_809.pdf subido con ID: 1vVtbMLbO--ZpbY8xZR97MK1u1UX7sN1U
Archivo AcuerdoRegistro_799.pdf subido con ID: 1U6XU8qLQJadeh5e2pZBhQ2e19shunP-L
Archivo AcuerdoRegistro_798.pdf subido con ID: 1S5dK2XyLMh09EBqYTD37PEjogngsxI74
Archivo AcuerdoRegistro_808.pdf subido con ID: 1VxDgcqBad4mAW4OGYE-neMRxeP2pd8yg
Archivo AcuerdoRegistro_811.pdf subido con ID: 1sVcmfS-FGBJVCHkfkJyPuyRc0Jjw_89q
Archivo AcuerdoRegistro_805.pdf subido con ID: 1HU9UnWlRrI2dH2olUj7ievNxb7vePQZh
Archivo AcuerdoRegistro_795.pdf subido con ID: 1OCJhGtRv9dnK_Dzaqs8g9BVZHoYW9Dbq
Archivo AcuerdoRegistro_794.pdf subido con ID: 1asPQIxAfSgxeGfaDck5K3i7C8b2D5Mfm
Archivo AcuerdoRegistro_804.pdf subido con ID: 1mVSlZiqzu1h8CuQ0sNfc5ajg201S8B4Y
Archivo AcuerdoRegistro_810.pdf subido con ID: 1EVrmLuXyFhUm5x7748Qh2DWZ9ME-bMRd
Archivo AcuerdoRegistro_806.

# Subir datos a Google Sheets Padrón_Víctimas_CDMX

In [78]:
# Se agregan las credenciales

key_path = "/path/to/your/keyfile.json"
sheets_scope = ["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]
creds = Credentials.from_service_account_file(key_path, scopes=sheets_scope)
gc = gspread.authorize(creds)
spreadsheet_url = "https://docs.google.com/spreadsheets/d/.../edit?gid=..."
sh = gc.open_by_url(spreadsheet_url)
worksheet = sh.worksheet("Padrón_Víctimas_CDMX")

In [79]:
# Se renombran columnas, ajusta formato y seleccionan las de interés

data = data.rename(columns={
    "tipo_registro": "ReconocimientoCalidadVíctima",
    "nombre_víctima": "NombreVíctima",
    "número_registro": "NúmeroRegistro",
    "fecha_registro": "FechaRegistro",
    "Alfanúmericaregistro": "AlfanúmericaRegistro",
    "numero_registro": "NúmeroRegistro",
    "directa_indirecta": "TipoVíctima",
    "carpeta_investigacion": "CarpetaInvestigación",
    "nombre_recomendacion": "NombreRecomendacion",
    "tipo_delito": "TipoDelito.ViolaciónDH",
    "derechos_violados": "DerechosHumanosViolados",
    "clave_víctima": "ClaveVíctimaRecomendación",
    "expediente_judicial": "ExpedienteJudicial"
})

data["numero_recomendacion"] = data["numero_recomendacion"].apply(
    lambda x: f"Recomendación {x}" if pd.notna(x) else x
)

data.loc[pd.notna(data["numero_recomendacion"]), "TipoDelito.ViolaciónDH"] = data["numero_recomendacion"]

data["TipoVíctima"] = data["TipoVíctima"].str.capitalize()
data["ClaveVíctimaRecomendación"] = data["ClaveVíctimaRecomendación"].str.capitalize()

data = data.iloc[:, [0, 2, 3, 4, 5, 6, 8, 10, 11, 13, 14, 16, 17]]

In [80]:
# Se agregan nuevas columnas

data["Año"] = "2024"

data["PostMortem"] = np.nan
data["AlcaldíaHechoVictimizante"] = np.nan
data["NNA"] = data["NombreVíctima"].apply(lambda x: 1 if "N.N.A." in str(x) else 0)
data["Sexo"] = np.nan
data["Teléfono"] = np.nan
data["CorreoElectrónico"] = np.nan
data["GAP"] = np.nan
data["CURP"] = np.nan
data["TiempoModoLugar"] = np.nan
data["Parentesco"] = np.nan

data = data.fillna("NA")

In [81]:
# Se establece el idioma español

locale.setlocale(locale.LC_TIME, "es_ES")

'es_ES'

In [82]:
# Se convierten datos vacíos en NA y se modifican columnas adicionales

data["FechaRegistro"] = pd.to_datetime(data["FechaRegistro"], format="%d de %B de %Y", errors="coerce", dayfirst=True)
data["FechaRegistro"] = pd.to_datetime(data["FechaRegistro"]).dt.strftime("%d/%m/%Y")


In [83]:
# Se vuelve a establecer el idioma default

locale.setlocale(locale.LC_ALL, "")

'C/UTF-8/C/C/C/C'

In [84]:
# Se agrega un patrón para incorporar a Google Sheets

for idx, row in data.iterrows():
    new_row = [
        str(row[column]) if column in data.columns else ""
        for column in worksheet.row_values(1)
    ]
    worksheet.append_row(new_row)
    time.sleep(1)
    
print("Datos agregados exitosamente a Google Sheets según el mapeo especificado.")

Datos agregados exitosamente a Google Sheets según el mapeo especificado.


# Automatizar el envío de emails

In [85]:
# Se define alcance

SCOPES = ["https://www.googleapis.com/auth/gmail.compose"]

In [86]:
# Se define la función para autentificar en gmail

from google.oauth2.credentials import Credentials

def authenticate_gmail():
    creds = None
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("Llaves/oauth.json", SCOPES)
            creds = flow.run_local_server(port=0)
        with open("token.json", "w") as token:
            token.write(creds.to_json())
    service = build("gmail", "v1", credentials=creds)
    return service

In [87]:
# Se define la función para crear el borrador
def create_draft(service, user_id, to_email, subject, body_text, attachments=None):
    message = email.mime.multipart.MIMEMultipart()
    message["to"] = to_email
    message["subject"] = subject

    msg_body = email.mime.text.MIMEText(body_text, "html")
    message.attach(msg_body)

    if attachments:
        for attachment in attachments:
            with open(attachment, "rb") as f:
                attach_part = email.mime.base.MIMEBase("application", "octet-stream")
                attach_part.set_payload(f.read())
                email.encoders.encode_base64(attach_part)
                attach_part.add_header(
                    "Content-Disposition",
                    f"attachment; filename={os.path.basename(attachment)}",
                )
                message.attach(attach_part)

    raw_message = base64.urlsafe_b64encode(message.as_bytes()).decode("ascii")

    draft = service.users().drafts().create(
        userId=user_id,
        body={"message": {"raw": raw_message}},
    ).execute()

    print(f"Draft id: {draft["id"]}")
    return draft


In [88]:
# Extraer números de patrones en la columna 'número_registro'
def extract_numbers_from_column(text):
    numbers = re.findall(r"CEAVI/RELOVI/(\d+)/2024", text)
    return numbers

In [89]:
# Se definen los df para cada tipo de de email

gmail_delitos = data1[data1["tipo_registro"] == "Delito – MP"]
gmail_judiciales = data1[data1["tipo_registro"] == "Delito – Judicial"]
gmail_dh = data1[data1["tipo_registro"] == "Violación DH"]

In [90]:
# Se crea el borrador en gmail para delitos MP

def main():
    service = authenticate_gmail()

    oficios_folder = "/path/to/your/acuerdos_registro/PDF_Oficios"
    acuerdos_folder = "/path/to/your/PDF_Acuerdos_Victimas"

    for _, row in gmail_delitos.iterrows():
        to_email = row["correo_electrónico"]
        subject = f"Se remite oficio CEAVICDMX/DFV/RELOVI/{row["número_oficio"]}/2024"
        body_text = f"""
        <html>
        <body style="font-family: 'Source Sans Pro', sans-serif; color:gray; font-size:11pt;">
        <p style="text-align: right;">Ciudad de México a {row["fecha_oficio"]}</p>
        <p style="text-align: right; font-weight: bold;">CEAVICDMX/DFVCDMX/RELOVI/{row["número_oficio"]}/2024</p>        
        <p style="text-align: right;"><strong>Nombres:</strong> {row["nombres_víctimas"]}</p>
        <p style="text-align: right;"><strong>Acuerdos:</strong> {row["número_registro"]}</p>
        <p style="text-align: right;"><strong>Carpeta de Investigación:</strong> {row["número_carpeta"]}</p>
        <p style="text-align: left; font-weight: bold;">{row["nombre_agente"]}</p>        
        <p style="text-align: left; font-weight: bold;">{row["cargo_agente"]}</p>        
        <p style="text-align: left; font-weight: bold;">{row["fiscalía_especializada"]}</p>   
        <p style="text-align: left; font-weight: bold;">P R E S E N T E</p>     
        <p style="text-align: justify;">Me dirijo a usted en atención a las solicitudes mediante las que gestiona la inscripción al Registro Local de Víctimas de las personas citadas al rubro, para lo cual me permito manifestarle lo siguiente:</p>
        <p style="text-align: justify;">Anexo al presente se encuentran los acuerdos originales de las personas citadas al rubro.</p>
        <p style="text-align: justify;">Con fundamento en lo dispuesto en el artículo 102 de la Ley General de Víctimas y el artículo 142, fracción XIII de la Ley de Víctimas para la Ciudad de México, le envío los acuerdos antes mencionados a fin de que se le haga del conocimiento a quien en derecho corresponda. De la misma manera, le informo que se reserva la remisión del presente expediente al Comité Interdisciplinario Evaluador de la Comisión Ejecutiva de Atención a Víctimas de la Ciudad de México, <u>hasta en tanto exista resolución judicial firme o bien, exista determinación del Ministerio Público que determine compensación subsidiaria</u>, esto último de conformidad con lo dispuesto por los artículos 66, 67, 68 y 69 de la Ley General de Víctimas en armonía con los numerales 63, 64, 65, 66 y 135 de la Ley de Víctimas para la Ciudad de México.</p>
        <p style="text-align: justify;">Finalmente, se hace de su conocimiento que la información contenida en el presente oficio, así como en los documentos anexos, contiene datos personales, por lo que su difusión es responsabilidad de quien los transmite y quien los recibe, en términos de las disposiciones vigentes en materia de datos personales, acceso a la información pública, transparencia y rendición de cuentas.</p>
        <p style="text-align: justify;">Sin más por el momento, agradezco de antemano la atención sobre el presente asunto.</p>
        <p style="text-align: left; font-weight: bold;">A T E N T A M E N T E</p>  
        <p style="text-align: left; font-weight: bold;">LIC. ...</p>             
        <p style="text-align: left; font-weight: bold;">COORDINADOR DEL ...</p>             
        <p style="text-align: left; font-weight: bold;">COMISIÓN EJECUTIVA DE ATENCIÓN ...</p>                     
        <p style="font-size:9pt;">BMCH</p>
        <div>
            <p style="text-align: justify; color: #666666; font-size:9pt;">
                De conformidad con el artículo 97 de la Ley de Austeridad, Transparencia en Remuneraciones, Prestaciones y Ejercicio de Recursos en la Ciudad de México las comunicaciones oficiales entre los entes y personas servidoras públicas de la Ciudad, así como la prestación de los servicios cuya naturaleza lo permita, <b>deberán hacerse preferentemente de manera electrónica, informática o telemática</b>; con la finalidad de evitar en la medida de los posible la utilización de papel; insumos de reproducción física de documentos, servicios de mensajería, pasajes, utilización de vehículos y sus combustibles. Esto, en correlación con el artículo 5 de la Ley de Ciudadanía Digital de la Ciudad de México que reconoce el derecho de las Personas para relacionarse y comunicarse con la Administración Pública y las Alcaldías, mediante el uso de medios electrónicos y tecnologías de la información y comunicación de uso generalizado. Por consiguiente, <b><u>la presente comunicación tiene carácter oficial pleno</u></b>.
            </p>
            <hr style="border: 1px solid #666666;">
            <p style="font-weight: bold; color: #444444; font-size:11pt;">Edgar Alejandro Gómez Jaimes | Coordinador del Registro Local de Víctimas</p>
            <p style="color: #999999; font-size:11pt;">Comisión Ejecutiva de Atención a Víctimas</p>
            <p><a href="mailto:...@...mx" style="color: blue; font-size:11pt;">reviceavi@...mx</a></p>
            <p><b>Tel.</b> 55 9131 4694</p>
            <p>Dirección: ..., Alcaldía Cuauhtémoc C.P. 06000, CDMX</p>
            <blockquote style="text-align: justify; font-size:9pt; border-left: 1px solid #cccccc; padding-left: 10px; color: #444444;">
                <i>“La información contenida en este correo, así como la contenida en los documentos anexos, puede contener datos personales, por lo que su difusión es responsabilidad de quien los transmite y quien los recibe, en términos de lo dispuesto por los artículos 6 fracciones XII, XXII, XXIII; 169, 186 y demás aplicables de la Ley de Transparencia, Acceso a la Información Pública y Rendición de Cuentas de la Ciudad de México. Los Datos Personales se encuentran protegidos por la Ley de Protección de Datos Personales en Posesión de Sujetos Obligados de la Ciudad de México, por lo que su difusión se encuentra tutelada en sus artículos 3 fracciones IX, XXVIII, XXIX, XXXIV, XXXVI, 9, 16, 25, 26, 37, 41, 46, 49, 50, de la Ley de Protección de Datos Personales en Posesión de Sujetos Obligados de la Ciudad de México y demás relativos y aplicables; debiendo sujetarse en su caso, a las disposiciones relativas a la creación, modificación o supresión de datos personales previstos. Asimismo, deberá estarse a lo señalado en los artículos 7, 8, 9, 10, 11 y demás aplicables de los Lineamientos Generales sobre Protección de Datos Personales en Posesión de Sujetos Obligados de la Ciudad de México." El presente correo y sus anexos pueden ser objeto de solicitudes de información pública, si usted no es el destinatario del mismo, por favor repórtelo inmediatamente al remitente del correo y bórrelo.”</i>
            </blockquote>
            <b style="font-size:10pt">
              <i>
                <span lang="ES" style="text-align: justify; font-size:9pt;color:rgb(68,68,68)">
            Queda prohibida cualquier remisión, retransmisión u otro uso de este correo, por personas o entidades distintas al destinatario legítimo.&nbsp;
                </span>
              </i>
            </b>
        </div>
        </body>
        </html>
        """

        oficios_files = [os.path.join(oficios_folder, f"OficioRELOVI_{row["número_oficio"]}.pdf")]
        
        numbers = extract_numbers_from_column(row["número_registro"])
        acuerdos_files = [os.path.join(acuerdos_folder, f"AcuerdoRegistro_{num}.pdf") for num in numbers]

        attachments = oficios_files + acuerdos_files

        create_draft(service, "me", to_email, subject, body_text, attachments)

if __name__ == "__main__":
    main()


Draft id: r7718323833249473154
Draft id: r-142647637683693327
Draft id: r-1191127161739347285
Draft id: r3649759124155026391
Draft id: r-75275989381193454
Draft id: r-5349148240239286832
Draft id: r7471251834324807388


In [91]:
# Se crea el borrador en gmail para jueces

def main():
    service = authenticate_gmail()

    oficios_folder = "/path/to/your/acuerdos_registro/PDF_Oficios"
    acuerdos_folder = "/path/to/your/PDF_Acuerdos_Victimas"

    for _, row in gmail_judiciales.iterrows():
        to_email = row["correo_electrónico"]
        subject = f"Se remite oficio CEAVICDMX/DFV/RELOVI/{row["número_oficio"]}/2024"
        body_text = f"""
        <html>
        <body style="font-family: 'Source Sans Pro', sans-serif; color:gray; font-size:11pt;">
        <p style="text-align: right;">Ciudad de México a {row["fecha_oficio"]}</p>
        <p style="text-align: right; font-weight: bold;">CEAVICDMX/DFVCDMX/RELOVI/{row["número_oficio"]}/2024</p>        
        <p style="text-align: right;"><strong>Nombres:</strong> {row["nombres_víctimas"]}</p>
        <p style="text-align: right;"><strong>Acuerdos:</strong> {row["número_registro"]}</p>
        <p style="text-align: right;"><strong>Carpeta Judicial:</strong> {row["expediente_judicial"]}</p>
        <p style="text-align: left; font-weight: bold;">{row["nombre_juez"]}</p>        
        <p style="text-align: left; font-weight: bold;">{row["cargo_juez"]}</p>        
        <p style="text-align: left; font-weight: bold;">P R E S E N T E</p>     
        <p style="text-align: justify;">Me dirijo a usted en atención a las solicitudes mediante las que gestiona la inscripción al Registro Local de Víctimas de las personas citadas al rubro, para lo cual me permito manifestarle lo siguiente:</p>
        <p style="text-align: justify;">Anexo al presente se encuentran los acuerdos originales de las personas citadas al rubro.</p>
        <p style="text-align: justify;">Con fundamento en lo dispuesto en el artículo 102 de la Ley General de Víctimas y el artículo 142, fracción XIII de la Ley de Víctimas para la Ciudad de México, le remito los acuerdos antes mencionados a fin de que se le haga del conocimiento a quien en derecho corresponda. De igual forma, le informo que el expediente será turnado al Comité Interdisciplinario Evaluador, de conformidad con el artículo 33, primer párrafo del Reglamento de la Ley de Víctimas para la Ciudad de México, una vez que se cuente con la respectiva constancia de notificación.</p>
        <p style="text-align: justify;">Finalmente, se hace de su conocimiento que la información contenida en el presente oficio, así como en los documentos anexos, contiene datos personales, por lo que su difusión es responsabilidad de quien los transmite y quien los recibe, en términos de las disposiciones vigentes en materia de datos personales, acceso a la información pública, transparencia y rendición de cuentas.</p>
        <p style="text-align: justify;">Sin más por el momento, agradezco de antemano la atención sobre el presente asunto.</p>
        <p style="text-align: left; font-weight: bold;">A T E N T A M E N T E</p>  
        <p style="text-align: left; font-weight: bold;">LIC. ...</p>             
        <p style="text-align: left; font-weight: bold;">COORDINADOR DEL ...</p>             
        <p style="text-align: left; font-weight: bold;">COMISIÓN EJECUTIVA DE ...</p>                     
        <p style="font-size:9pt;">BMCH</p>
        <div>
            <p style="text-align: justify; color: #666666; font-size:9pt;">
                De conformidad con el artículo 97 de la Ley de Austeridad, Transparencia en Remuneraciones, Prestaciones y Ejercicio de Recursos en la Ciudad de México las comunicaciones oficiales entre los entes y personas servidoras públicas de la Ciudad, así como la prestación de los servicios cuya naturaleza lo permita, <b>deberán hacerse preferentemente de manera electrónica, informática o telemática</b>; con la finalidad de evitar en la medida de los posible la utilización de papel; insumos de reproducción física de documentos, servicios de mensajería, pasajes, utilización de vehículos y sus combustibles. Esto, en correlación con el artículo 5 de la Ley de Ciudadanía Digital de la Ciudad de México que reconoce el derecho de las Personas para relacionarse y comunicarse con la Administración Pública y las Alcaldías, mediante el uso de medios electrónicos y tecnologías de la información y comunicación de uso generalizado. Por consiguiente, <b><u>la presente comunicación tiene carácter oficial pleno</u></b>.
            </p>
            <hr style="border: 1px solid #666666;">
            <p style="font-weight: bold; color: #444444; font-size:11pt;">Edgar Alejandro Gómez Jaimes | Coordinador del Registro Local de Víctimas</p>
            <p style="color: #999999; font-size:11pt;">Comisión Ejecutiva de Atención a Víctimas</p>
            <p><a href="mailto:reviceavi@...mx" style="color: blue; font-size:11pt;">reviceavi@...mx</a></p>
            <p><b>Tel.</b> 55 9131 4694</p>
            <p>Dirección: ..., Alcaldía Cuauhtémoc C.P. 06000, CDMX</p>
            <blockquote style="text-align: justify; font-size:9pt; border-left: 1px solid #cccccc; padding-left: 10px; color: #444444;">
                <i>“La información contenida en este correo, así como la contenida en los documentos anexos, puede contener datos personales, por lo que su difusión es responsabilidad de quien los transmite y quien los recibe, en términos de lo dispuesto por los artículos 6 fracciones XII, XXII, XXIII; 169, 186 y demás aplicables de la Ley de Transparencia, Acceso a la Información Pública y Rendición de Cuentas de la Ciudad de México. Los Datos Personales se encuentran protegidos por la Ley de Protección de Datos Personales en Posesión de Sujetos Obligados de la Ciudad de México, por lo que su difusión se encuentra tutelada en sus artículos 3 fracciones IX, XXVIII, XXIX, XXXIV, XXXVI, 9, 16, 25, 26, 37, 41, 46, 49, 50, de la Ley de Protección de Datos Personales en Posesión de Sujetos Obligados de la Ciudad de México y demás relativos y aplicables; debiendo sujetarse en su caso, a las disposiciones relativas a la creación, modificación o supresión de datos personales previstos. Asimismo, deberá estarse a lo señalado en los artículos 7, 8, 9, 10, 11 y demás aplicables de los Lineamientos Generales sobre Protección de Datos Personales en Posesión de Sujetos Obligados de la Ciudad de México." El presente correo y sus anexos pueden ser objeto de solicitudes de información pública, si usted no es el destinatario del mismo, por favor repórtelo inmediatamente al remitente del correo y bórrelo.”</i>
            </blockquote>
            <b style="font-size:10pt">
              <i>
                <span lang="ES" style="text-align: justify; font-size:9pt;color:rgb(68,68,68)">
            Queda prohibida cualquier remisión, retransmisión u otro uso de este correo, por personas o entidades distintas al destinatario legítimo.&nbsp;
                </span>
              </i>
            </b>
        </div>
        </body>
        </html>
        """

        oficios_files = [os.path.join(oficios_folder, f"OficioRELOVI_{row["número_oficio"]}.pdf")]
        
        numbers = extract_numbers_from_column(row["número_registro"])
        acuerdos_files = [os.path.join(acuerdos_folder, f"AcuerdoRegistro_{num}.pdf") for num in numbers]

        attachments = oficios_files + acuerdos_files

        create_draft(service, "me", to_email, subject, body_text, attachments)

if __name__ == "__main__":
    main()


Draft id: r8291563226079120117


In [ ]:
# Se crea el borrador en violaciones a los DH

def main():
    service = authenticate_gmail()

    oficios_folder = "/path/to/your/acuerdos_registro/PDF_Oficios"
    acuerdos_folder = "/path/to/your/PDF_Acuerdos_Victimas"

    for _, row in gmail_dh.iterrows():
        to_email = row["correo_electrónico"]
        subject = f"Se remite oficio CEAVICDMX/DFV/RELOVI/{row["número_oficio"]}/2024"
        body_text = f"""
        <html>
        <body style="font-family: 'Source Sans Pro', sans-serif; color:gray; font-size:11pt;">
        <p style="text-align: right;">Ciudad de México a {row["fecha_oficio"]}</p>
        <p style="text-align: right; font-weight: bold;">CEAVICDMX/DFVCDMX/RELOVI/{row["número_oficio"]}/2024</p>        
        <p style="text-align: right;"><strong>Nombres:</strong> {row["nombres_víctimas"]}</p>
        <p style="text-align: right;"><strong>Acuerdos:</strong> {row["número_registro"]}</p>
        <p style="text-align: right;"><strong>Recomendación:</strong> {row["número_recomendación"]}</p>
        <p style="text-align: left; font-weight: bold;">MTRA. ...</p>     
        <p style="text-align: left; font-weight: bold;">DIRECTORA ...</p>     
        <p style="text-align: left; font-weight: bold;">P R E S E N T E</p>     
        <p style="text-align: justify;">Me dirijo a usted en atención a las solicitudes mediante las que gestiona la inscripción al Registro Local de Víctimas de las personas citadas al rubro, para lo cual me permito manifestarle lo siguiente:</p>
        <p style="text-align: justify;">Anexo al presente se encuentran los acuerdos originales de las personas citadas al rubro.</p>
        <p style="text-align: justify;">Con fundamento en lo dispuesto en el artículo 102 de la Ley General de Víctimas y el artículo 142, fracción XIII de la Ley de Víctimas para la Ciudad de México, le envío los acuerdos antes mencionados a fin de que se le haga del conocimiento a quien en derecho corresponda. De la misma manera, le informo que se reserva la remisión del presente expediente al Comité Interdisciplinario Evaluador de la Comisión Ejecutiva de Atención a Víctimas de la Ciudad de México, <u>hasta en tanto exista resolución judicial firme o bien, exista determinación del Ministerio Público que determine compensación subsidiaria</u>, esto último de conformidad con lo dispuesto por los artículos 66, 67, 68 y 69 de la Ley General de Víctimas en armonía con los numerales 63, 64, 65, 66 y 135 de la Ley de Víctimas para la Ciudad de México.</p>
        <p style="text-align: justify;">Finalmente, se hace de su conocimiento que la información contenida en el presente oficio, así como en los documentos anexos, contiene datos personales, por lo que su difusión es responsabilidad de quien los transmite y quien los recibe, en términos de las disposiciones vigentes en materia de datos personales, acceso a la información pública, transparencia y rendición de cuentas.</p>
        <p style="text-align: justify;">Sin más por el momento, agradezco de antemano la atención sobre el presente asunto.</p>
        <p style="text-align: left; font-weight: bold;">A T E N T A M E N T E</p>  
        <p style="text-align: left; font-weight: bold;">LIC. ....</p>             
        <p style="text-align: left; font-weight: bold;">COORDINADOR DEL ...</p>             
        <p style="text-align: left; font-weight: bold;">COMISIÓN EJECUTIVA ...</p>                     
        <p style="font-size:9pt;">BMCH</p>
        <div>
            <p style="text-align: justify; color: #666666; font-size:9pt;">
                De conformidad con el artículo 97 de la Ley de Austeridad, Transparencia en Remuneraciones, Prestaciones y Ejercicio de Recursos en la Ciudad de México las comunicaciones oficiales entre los entes y personas servidoras públicas de la Ciudad, así como la prestación de los servicios cuya naturaleza lo permita, <b>deberán hacerse preferentemente de manera electrónica, informática o telemática</b>; con la finalidad de evitar en la medida de los posible la utilización de papel; insumos de reproducción física de documentos, servicios de mensajería, pasajes, utilización de vehículos y sus combustibles. Esto, en correlación con el artículo 5 de la Ley de Ciudadanía Digital de la Ciudad de México que reconoce el derecho de las Personas para relacionarse y comunicarse con la Administración Pública y las Alcaldías, mediante el uso de medios electrónicos y tecnologías de la información y comunicación de uso generalizado. Por consiguiente, <b><u>la presente comunicación tiene carácter oficial pleno</u></b>.
            </p>
            <hr style="border: 1px solid #666666;">
            <p style="font-weight: bold; color: #444444; font-size:11pt;">Edgar Alejandro Gómez Jaimes | Coordinador del Registro Local de Víctimas</p>
            <p style="color: #999999; font-size:11pt;">Comisión Ejecutiva de Atención a Víctimas</p>
            <p><a href="mailto:reviceavi@...mx" style="color: blue; font-size:11pt;">reviceavi@...mx</a></p>
            <p><b>Tel.</b> 55 9131 4694</p>
            <p>Dirección: ..., Alcaldía Cuauhtémoc C.P. 06000, CDMX</p>
            <blockquote style="text-align: justify; font-size:9pt; border-left: 1px solid #cccccc; padding-left: 10px; color: #444444;">
                <i>“La información contenida en este correo, así como la contenida en los documentos anexos, puede contener datos personales, por lo que su difusión es responsabilidad de quien los transmite y quien los recibe, en términos de lo dispuesto por los artículos 6 fracciones XII, XXII, XXIII; 169, 186 y demás aplicables de la Ley de Transparencia, Acceso a la Información Pública y Rendición de Cuentas de la Ciudad de México. Los Datos Personales se encuentran protegidos por la Ley de Protección de Datos Personales en Posesión de Sujetos Obligados de la Ciudad de México, por lo que su difusión se encuentra tutelada en sus artículos 3 fracciones IX, XXVIII, XXIX, XXXIV, XXXVI, 9, 16, 25, 26, 37, 41, 46, 49, 50, de la Ley de Protección de Datos Personales en Posesión de Sujetos Obligados de la Ciudad de México y demás relativos y aplicables; debiendo sujetarse en su caso, a las disposiciones relativas a la creación, modificación o supresión de datos personales previstos. Asimismo, deberá estarse a lo señalado en los artículos 7, 8, 9, 10, 11 y demás aplicables de los Lineamientos Generales sobre Protección de Datos Personales en Posesión de Sujetos Obligados de la Ciudad de México." El presente correo y sus anexos pueden ser objeto de solicitudes de información pública, si usted no es el destinatario del mismo, por favor repórtelo inmediatamente al remitente del correo y bórrelo.”</i>
            </blockquote>
            <b style="font-size:10pt">
              <i>
                <span lang="ES" style="text-align: justify; font-size:9pt;color:rgb(68,68,68)">
            Queda prohibida cualquier remisión, retransmisión u otro uso de este correo, por personas o entidades distintas al destinatario legítimo.&nbsp;
                </span>
              </i>
            </b>
        </div>
        </body>
        </html>
        """

        oficios_files = [os.path.join(oficios_folder, f"OficioRELOVI_{row["número_oficio"]}.pdf")]
        
        numbers = extract_numbers_from_column(row["número_registro"])
        acuerdos_files = [os.path.join(acuerdos_folder, f"AcuerdoRegistro_{num}.pdf") for num in numbers]

        attachments = oficios_files + acuerdos_files

        create_draft(service, "me", to_email, subject, body_text, attachments)

if __name__ == "__main__":
    main()
